In [1]:
import cv2
import numpy as np
import random
import os
from PIL import Image
import time
import os
import smtplib
from pyfirmata import Arduino

In [2]:

board = Arduino('COM5')
time.sleep(2.0)
led = board.get_pin('d:8:o')

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
distance_thres = 50

# is_human_detected = False
count = 0
count_1 = 0

my_email = 'Your email'
my_pass = 'your password'

cap = cv2.VideoCapture(0)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

_,frame = cap.read()
fourcc = cv2.VideoWriter_fourcc(*"MJPG")
writer = cv2.VideoWriter('output.avi', fourcc, 30,(frame.shape[1], frame.shape[0]), True)
ret = True
while ret:
    ret, img = cap.read()
    if ret:
        height, width = img.shape[:2]
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)
        confidences = []
        boxes = []
        
        is_human_detected = False
        
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                if class_id!=0:
                    continue
                confidence = scores[class_id]
                if confidence > 0.3: # here
                    is_human_detected = True
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))

        print(f"-----> Human Detected: ", is_human_detected)
        count += 1

        if is_human_detected:
            led.write(0)
        else:
            led.write(1)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        persons = []
        person_centres = []
        for i in range(len(boxes)):
            if i in indexes:
                x,y,w,h = boxes[i]
                persons.append(boxes[i])
                person_centres.append([x+w//2,y+h//2])
        
        v = 0
        for (x,y,w,h) in persons:
            color = (0,255,0)
            cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
            cv2.circle(img,(x+w//2,y+h//2),2,(0,0,255),2)

            if count_1 == 0:
                ### Sending email alert that someone has entered office.
                with smtplib.SMTP("smtp.gmail.com") as connection:
                    connection.starttls()
                    connection.login(user=my_email, password=my_pass)
                    connection.sendmail(from_addr=my_email,
                            to_addrs="lakho75@yahoo.com",
                            msg="Subject: Person Alert\n\nSomeone has Entered the Office!")
                    print("Mail has been sent successfully!")  


            count_1 = count_1 + 1 


        writer.write(img)
        cv2.imshow("Image", img)
    
    if cv2.waitKey(1) == 27:
        break

if is_human_detected:
    print("Human was detected")
    led.write(1)

cap.release()
cv2.destroyAllWindows()

-----> Human Detected:  True
Mail has been sent successfully!
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
-----> Human Detected:  True
Human was detected
